#RNN

In [ ]:
# Import necessary libraries
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.datasets import imdb
from random import randint

# Load the IMDB dataset, limiting to the top 5000 most frequent words
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

# Pad sequences to ensure uniform input length
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

# Create the LSTM model
model = Sequential()
model.add(Embedding(5000, 128))          # Embedding layer for word embeddings
model.add(LSTM(128, activation='tanh', recurrent_activation='sigmoid'))  # LSTM layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification


# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_test, y_test), shuffle=True, verbose=1)

# Print model summary
model.summary()




# Predict on the test set
predictions = model.predict(x_test)

# Randomly select a review from the test set for evaluation
arr_ind = randint(0, len(x_test) - 1)  # Random index
index = imdb.get_word_index()           # Word index dictionary
reverse_index = dict([(value, key) for (key, value) in index.items()])  # Reverse dictionary for decoding
decoded = " ".join([reverse_index.get(i - 3, "#") for i in x_test[arr_ind]])  # Decode review text

# Determine the sentiment of the review
predicted_sentiment = "Positive" if predictions[arr_ind] >= 0.5 else "Negative"
actual_sentiment = "Positive" if y_test[arr_ind] == 1 else "Negative"

# Print results
print("Sentence:", decoded)
print("Review:", predicted_sentiment)
print("Predicted Value:", predictions[arr_ind][0])
print("Expected Value:", y_test[arr_ind])



#AUTOENCODER

In [ ]:
import numpy as np
import keras
from keras import layers
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)




input_img = keras.Input(shape=(784,))
encoded = layers.Dense(128, activation='sigmoid')(input_img)
decoded = layers.Dense(784, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

import matplotlib.pyplot as plt

encoded_imgs = autoencoder.predict(x_test)
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()



### ADDING SPARISTY

from keras import regularizers

encoding_dim = 32
input_img = keras.Input(shape=(784,))
# Add a Dense layer with a L1 activity regularizer
encoded = layers.Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = layers.Dense(784, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_img, decoded)

# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

autoencoder.fit(x_train, x_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# Use Matplotlib
import matplotlib.pyplot as plt

n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()




## DENOISING
#Denoising

from tensorflow.keras.layers import GaussianNoise

(x_train,_),(x_test,_) = mnist.load_data()
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
print(x_train.shape)


x_train=x_train.reshape((len(x_train),np.prod(x_train.shape[1:])))
x_test=x_test.reshape((len(x_test),np.prod(x_test.shape[1:])))
print(x_train.shape)

#Architechture

input_img = Input(shape=(784,))
#add noise to input
noisy_input = GaussianNoise(0.5)(input_img)

encoded = Dense(128,activation='sigmoid')(input_img)
decoded = Dense(784,activation='sigmoid')(encoded)

#Build autencoder model

autoencoder = Model(input_img,decoded)
#compile
autoencoder.compile(optimizer= Adam(),loss='binary_crossentropy')
#train
autoencoder.fit(x_train,x_train,epochs=5,batch_size=256,shuffle=True,validation_data=(x_test,x_test))





import matplotlib.pyplot as plt

#encode and decode some digits
encoded_imgs = autoencoder.predict(x_test)

# Generate noisy images
noise_factor = 0.5
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

n = 10  # How many digits we will display
plt.figure(figsize=(20, 6))
for i in range(n):
    # Display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display noisy
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(encoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


#OPTIMIZERS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate synthetic data
np.random.seed(42)
x = 2 * np.random.rand(100, 1)
y = 4 + 3 * x + np.random.randn(100, 1)

X_b = np.c_[np.ones((100, 1)), x]  # Add bias term



# Initialization for all methods
theta_sgd = np.random.randn(2, 1)
theta_batch = np.random.randn(2, 1)
theta_mini = np.random.randn(2, 1)
eta = 0.1
n_iterations = 10000
m = 100
batch_size = 20
# Stochastic Gradient Descent
def stochastic_gradient_descent(X_b, y, theta, eta, n_iterations, m):
  for iteration in range(n_iterations):
    for i in range(m):
      random_index = np.random.randint(m)
      xi = X_b[random_index:random_index + 1]
      yi = y[random_index:random_index + 1]
      gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
      theta = theta - eta * gradients
  return theta
# Batch Gradient Descent
def batch_gradient_descent(X_b, y, theta, eta, n_iterations):
  for iteration in range(n_iterations):
    gradients = 2 / m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients
  return theta
# Mini-Batch Gradient Descent
def mini_batch_gradient_descent(X_b, y, theta, eta, n_iterations, m, batch_size):
  for iteration in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, batch_size):
      xi = X_b_shuffled[i:i+batch_size]
      yi = y_shuffled[i:i+batch_size]
      gradients = 2 / batch_size * xi.T.dot(xi.dot(theta) - yi)
      theta = theta - eta * gradients
  return theta
# Run the optimization functions
theta_sgd = stochastic_gradient_descent(X_b, y, theta_sgd, eta, n_iterations, m)
theta_batch = batch_gradient_descent(X_b, y, theta_batch, eta, n_iterations)
theta_mini = mini_batch_gradient_descent(X_b, y, theta_mini, eta, n_iterations, m, batch_size)
# Print results
print("Stochastic Gradient Descent:", theta_sgd.flatten())
print("Batch Gradient Descent:", theta_batch.flatten())
print("Mini-Batch Gradient Descent:", theta_mini.flatten())




# Plot the results
plt.scatter(x, y, color="grey")
plt.xlabel("X")
plt.ylabel("Y")
plt.plot(x, X_b.dot(theta_sgd), label='Stochastic')
plt.plot(x, X_b.dot(theta_batch), label='Batch')
plt.plot(x, X_b.dot(theta_mini), label='Mini-Batch')
plt.legend()
plt.show()





## Implementing Adavance Optimizers


#Momentum
#Adagrad
#RMSprop

theta_momentum = np.random.randn(2, 1)
theta_adagrad = np.random.randn(2, 1)
theta_rmsprop = np.random.randn(2, 1)
eta = 0.1
n_iterations = 10000
m = 100

# Momentum-based Gradient Descent
def momentum_gradient_descent(X_b, y, theta, eta, n_iterations, m, beta=0.9):
  velocity = np.zeros_like(theta)
  for iteration in range(n_iterations):
    gradients = 2 / m * X_b.T.dot(X_b.dot(theta) - y)
    velocity = beta * velocity + (1 - beta) * gradients
    theta = theta - eta * velocity
  return theta

# Adagrad
def adagrad(X_b, y, theta, eta, n_iterations, m, epsilon=1e-7):
  s = np.zeros_like(theta)
  for iteration in range(n_iterations):
    gradients = 2 / m * X_b.T.dot(X_b.dot(theta) - y)
    s += gradients ** 2
    theta = theta - eta * gradients / (np.sqrt(s) + epsilon)
  return theta

# RMSprop
def rmsprop(X_b, y, theta, eta, n_iterations, m, beta=0.9, epsilon=1e-7):
  s = np.zeros_like(theta)
  for iteration in range(n_iterations):
    gradients = 2 / m * X_b.T.dot(X_b.dot(theta) - y)
    s = beta * s + (1 - beta) * gradients ** 2
    theta = theta - eta * gradients / (np.sqrt(s) + epsilon)
  return theta

theta_momentum = momentum_gradient_descent(X_b, y, theta_momentum, eta, n_iterations, m)
theta_adagrad = adagrad(X_b, y, theta_adagrad, eta, n_iterations, m)
theta_rmsprop = rmsprop(X_b, y, theta_rmsprop, eta, n_iterations, m)

# Print results
print("Momentum-based Gradient Descent:", theta_momentum.flatten())
print("Adagrad:", theta_adagrad.flatten())
print("RMSprop:", theta_rmsprop.flatten())



# Plot the results
plt.scatter(x, y, color="grey")
plt.xlabel("X")
plt.ylabel("Y")
plt.plot(x, X_b.dot(theta_momentum), label='Momentum')
plt.plot(x, X_b.dot(theta_adagrad), label='Adagrad')
plt.plot(x, X_b.dot(theta_rmsprop), label='RMSprop')
plt.legend()
plt.show()


#CNN USING FILTERS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imshow, imread
from skimage.color import rgb2yuv, rgb2hsv, rgb2gray, yuv2rgb, hsv2rgb
from scipy.signal import convolve2d

#Reading Image
dog = imread('/content/img.jpg')
plt.figure(num=None, figsize=(8,6), dpi=80)
imshow(dog)


#Filter MAtrices

sharpen = np.array([[0,-1,0],
                    [-1,5,-1],
                    [0,-1,0]])

blur = np.array([[0.11,0.11,0.11],
                    [0.11,0.11,0.11],
                    [0.11,0.11,0.11]])


vertical = np.array([[-1,0,1],
                    [-2,0,2],
                    [-1,0,1]])


gaussian = (1/16.0) * np.array([[1,2,1],
                                [2,4,2],
                                [1,2,1]])



#plotting the filters

fig,ax = plt.subplots(1,3, figsize = (17,10))
ax[0].imshow(sharpen, cmap='gray')
ax[0].set_title(f'Sharpen', fontsize=18)

ax[1].imshow(blur, cmap='gray')
ax[1].set_title(f'Blur', fontsize=18)

ax[2].imshow(vertical, cmap='gray')
ax[2].set_title(f'Vertical', fontsize=18)


#Grayscaling Image
dog_gray = rgb2gray(dog)
plt.figure(num=None, figsize=(8,6), dpi=80)
imshow(dog_gray)


#Function for applying filters
def multi_convolver(image, kernel, iterations):
  for i in range(iterations):
    image = convolve2d(image, kernel, 'same', boundary = 'fill', fillvalue = 0)
  return image

convolved_image = multi_convolver(dog_gray, sharpen, 1)

plt.figure(num=None, figsize=(8,6), dpi=80)
imshow(convolved_image);



#For colored Image
def convolver_rgb(image, kernel, iterations = 1):
  convolved_image_r = multi_convolver(image[:,:,0], kernel, iterations)
  convolved_image_g = multi_convolver(image[:,:,1], kernel, iterations)
  convolved_image_b = multi_convolver(image[:,:,2], kernel, iterations)

  reformed_image = np.dstack((np.rint(abs(convolved_image_r)),np.rint(abs(convolved_image_g)),np.rint(abs(convolved_image_b))))/255

  fig,ax = plt.subplots(1,3, figsize = (17,10))

  ax[0].imshow(abs(convolved_image_r), cmap='Reds')
  ax[0].set_title(f'Red', fontsize=15)

  ax[1].imshow(abs(convolved_image_g), cmap='Greens')
  ax[1].set_title(f'Green', fontsize=18)

  ax[2].imshow(abs(convolved_image_b), cmap='Blues')
  ax[2].set_title(f'Blue', fontsize=18)

  return np.array(reformed_image*255).astype(np.uint8)

#Can add different filters (defined above) here
convolved_rgb_gauss = convolver_rgb(dog, vertical.T ,1)




#CNN PARAMETER CALCULATION
from keras.models import Sequential
from keras.layers import Conv2D


model = Sequential()
model.add(Conv2D(32, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 padding='same',
                 use_bias=False))
model.add(Conv2D(17, (3,3), padding='same', use_bias=False))
model.add(Conv2D(13, (3,3), padding='same', use_bias=False))
model.add(Conv2D(7, (3,3), padding='same', use_bias=False))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()


model = Sequential()
model.add(Conv2D(32, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 padding='same',
                 use_bias=True))
model.add(Conv2D(17, (3,3), padding='same', use_bias=True))
model.add(Conv2D(13, (3,3), padding='same', use_bias=True))
model.add(Conv2D(7, (3,3), padding='same', use_bias=True))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()


model = Sequential()
model.add(Conv2D(32, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 use_bias=False))
model.add(Conv2D(17, (3,3), use_bias=False))
model.add(Conv2D(13, (3,3), use_bias=False))
model.add(Conv2D(7, (3,3), use_bias=False))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()



model = Sequential()
model.add(Conv2D(32, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 padding='valid',
                 use_bias=False))
model.add(Conv2D(17, (3,3), padding='valid', use_bias=False))
model.add(Conv2D(13, (3,3), padding='valid', use_bias=False))
model.add(Conv2D(7, (3,3), padding='valid', use_bias=False))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()


model = Sequential()
model.add(Conv2D(10, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 strides = (1,1),
                 padding='valid',
                 use_bias=False))
model.add(Conv2D(20, (5,5), (2,2), padding='valid', use_bias=False))
model.add(Conv2D(40, (5,5), (2,2), padding='valid', use_bias=False))


model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()



model = Sequential()
model.add(Conv2D(10, input_shape=(39,39,3),
                 kernel_size = (3,3),
                 padding='valid',))
model.add(Conv2D(20, (5,5), (2,2), padding='valid' ))
model.add(Conv2D(40, (5,5), (2,2),padding='valid'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()




# MAX POLLING
import tensorflow as tf
x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.],
                 [7., 8., 9.]])
x = tf.reshape(x, [1, 3, 3, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')
max_pool_2d(x)



x = tf.constant([[1., 2., 3., 4.],
                 [5., 6., 7., 8.],
                 [9., 10., 11., 12.]])
x = tf.reshape(x, [1, 3, 4, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
   strides=(2, 2), padding='valid')
max_pool_2d(x)


x = tf.constant([[1., 2., 3., 4.],
                 [5., 6., 7., 8.],
                 [9., 10., 11., 12.]])
x = tf.reshape(x, [1, 3, 4, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
   strides=(2, 2), padding='same')
max_pool_2d(x)


x = tf.constant([[1., 2., 3., 4.],
                 [5., 6., 7., 8.],
                 [9., 10., 11., 12.]])
x = tf.reshape(x, [1, 3, 4, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
   strides=(1, 1), padding='same')
max_pool_2d(x)

#REGULARIZATION

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
import warnings
from mlxtend.plotting import plot_decision_regions
from matplotlib.colors import ListedColormap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles,make_moons



np.random.seed(800)
X,y=make_moons(n_samples=100, noise=0.2, random_state=1)
zero_one_colourmap=ListedColormap(('red','green'))
rcParams['figure.figsize']=14,7
plt.scatter(X[:,0],X[:,1],
c=y, s=100,
cmap=zero_one_colourmap)
plt.show()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = Sequential()
model.add(Dense(500, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=1000, verbose=0)

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

reg_model = Sequential()
reg_model.add(Dense(500, input_dim=2, activation='relu', kernel_regularizer='l2'))
reg_model.add(Dense(1, activation='sigmoid', kernel_regularizer='l2'))
reg_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

reg_history = reg_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, verbose=0)

plt.plot(reg_history.history['loss'], label='train')
plt.plot(reg_history.history['val_loss'], label='test')
plt.legend()
plt.show()




#AUGUMENTATION

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255  # Reshape and normalize
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical", input_shape=(28, 28, 1)),  # Specify input shape here
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])

# Create a simple CNN model
def create_cnn_model():
    model = models.Sequential()
    model.add(data_augmentation)  # Include data augmentation in the model
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer

    return model

# Create and compile the model
model = create_cnn_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with data augmentation
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='test accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

#DROP OUT

In [ ]:
# DropOut with CNN on Mnist dataset

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Build CNN model with Dropout
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout layer with 50% rate
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

#MINIST DATASET FILTER CNN

In [ ]:
# Filter on Mnist using CNN

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255  # Reshape and normalize
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Define the CNN model
def create_cnn_model():
    model = models.Sequential()

    # Convolutional layer with 3x3 filters (Sharpening Filter)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

    # Convolutional layer with 5x5 filters (Blur Filter)
    model.add(layers.Conv2D(64, (5, 5), activation='relu'))

    # Pooling layer
    model.add(layers.MaxPooling2D((2, 2)))

    # Another convolutional layer (Edge Detection Filter)
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))

    # Flattening layer
    model.add(layers.Flatten())

    # Fully connected layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.Dense(10, activation='softmax'))  # Output layer

    return model

# Create and compile the model
model = create_cnn_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='test accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

#BINARY CLASSIFICATION

In [ ]:
# Binary Class
import numpy as np

def sigmoid(x):
  return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
  return x * (1 - x)

def mean_squared_error_loss(y_true, y_pred):
  return np.mean(np.power(y_true - y_pred, 2))

## input
input_data = np.array([[0,0], [0,1], [1,0], [1,1]])
output_data = np.array([[0], [1], [1], [0]])

## Traning Parameter
np.random.seed(42)
input_size = 2
hidden_size = 2
output_size = 1

## Random Weights & Bias
weights_input_to_hidden = np.random.rand(input_size, hidden_size)
bias_hidden = np.random.rand(hidden_size)

weights_hidden_to_output = np.random.rand(hidden_size, output_size)
bias_output = np.random.rand(output_size)

#Training Parameter
learning_rate = 0.1
epochs = 10000


for epoch in range(epochs):
  #forward pass
  hidden_input = np.dot(input_data, weights_input_to_hidden)+bias_hidden
  hidden_output = sigmoid(hidden_input)

  final_input = np.dot(hidden_output, weights_hidden_to_output)+bias_output
  final_output = sigmoid(final_input)


  #computing loss
  loss = mean_squared_error_loss(output_data, final_output)

  #backpropagation
  error = final_output-output_data
  gradient_output = error*sigmoid_derivative(final_output)

  # hidden layer eror and gradient
  error_hidden = gradient_output.dot(weights_hidden_to_output.T)
  gradient_hidden = error_hidden*sigmoid_derivative(hidden_output)

  #updatingweight and biases
  weights_hidden_to_output-=learning_rate*np.dot(hidden_output.T, gradient_output)
  bias_output-=learning_rate*np.mean(gradient_output, axis=0)

  weights_input_to_hidden-=learning_rate*np.dot(input_data.T, gradient_hidden)
  bias_hidden-=learning_rate*np.mean(gradient_hidden, axis=0)

  #print 1000 loss
  if epoch%1000==0:
    print(f'Epoch: {epoch}, Loss: {loss}')

#compute out for each input pair after training
result = []
for input_pair in input_data:
  hidden_input = np.dot(input_pair, weights_input_to_hidden)+bias_hidden
  hidden_output = sigmoid(hidden_input)
  final_input = np.dot(hidden_output, weights_hidden_to_output)+bias_output
  final_output = sigmoid(final_input)
  result.append(final_output)
  print(f'Input: {input_pair}, Output: {final_output}')



#MULTICLASSIFICATION

In [ ]:
import numpy as np

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def softmax(x):
    exps = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exps / np.sum(exps, axis=-1, keepdims=True)

def categorical_crossentropy_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred))

# Input data and output data
input_data = np.array([[0.8, 0.6, 0.7]])
output_data = np.array([[0, 1, 0]])

# Initialize weights and biases
weights_input_to_hidden = np.array([[0.2, 0.4, 0.1],
                                    [0.5, 0.3, 0.2],
                                    [0.3, 0.7, 0.8]])

bias_hidden = np.array([0.1, 0.2, 0.3])

weights_hidden_to_output = np.array([[0.6, 0.4, 0.5],
                                     [0.1, 0.2, 0.3],
                                     [0.3, 0.7, 0.2]])

bias_output = np.array([0.1, 0.2, 0.3])

# Learning rate
learning_rate = 0.01

# Training loop
epochs = 1000
for epoch in range(epochs):
    # Forward Pass
    hidden_input = np.dot(input_data, weights_input_to_hidden) + bias_hidden
    hidden_output = relu(hidden_input)

    final_input = np.dot(hidden_output, weights_hidden_to_output) + bias_output
    final_output = softmax(final_input)

    # Compute loss
    loss = categorical_crossentropy_loss(output_data, final_output)

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss {loss:.4f}")

    # Backpropagation
    error = final_output - output_data

    # Gradient for weights_hidden_to_output and bias_output
    gradient_weights_hidden_to_output = np.dot(hidden_output.T, error)
    gradient_bias_output = np.sum(error, axis=0, keepdims=True)

    # Propagate the error back to the hidden layer
    error_hidden = np.dot(error, weights_hidden_to_output.T)
    error_hidden *= relu_derivative(hidden_input)

    # Gradient for weights_input_to_hidden and bias_hidden
    gradient_weights_input_to_hidden = np.dot(input_data.T, error_hidden)
    gradient_bias_hidden = np.sum(error_hidden, axis=0, keepdims=True)

    # Update weights and biases using gradient descent
    weights_hidden_to_output -= learning_rate * gradient_weights_hidden_to_output
    bias_output -= learning_rate * gradient_bias_output.squeeze()
    weights_input_to_hidden -= learning_rate * gradient_weights_input_to_hidden
    bias_hidden -= learning_rate * gradient_bias_hidden.squeeze()

# Print final loss after training
print(f"Final Loss: {loss:.4f}")
print(f"Input: {input_data}, Output : {final_output}")

#brain tumor detection


In [ ]:
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


from google.colab import drive
drive.mount('/content/drive')



encoder = OneHotEncoder()
encoder.fit([[0], [1]])

# 0 - Tumor
# 1 - Normal


import zipfile
import os


zip_train=zipfile.ZipFile('/content/drive/MyDrive/brain_tumor_dataset.zip', 'r')
zip_train.extractall('/content/drive/MyDrive')
zip_train.close()




# This cell updates result list for images with tumor
data = []
paths = []
result = []

for r, d, f in os.walk(r'/content/drive/MyDrive/brain_tumor_dataset/yes'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[0]]).toarray())


# This cell updates result list for images with tumor

paths = []
for r, d, f in os.walk(r"/content/drive/MyDrive/brain_tumor_dataset/no"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[1]]).toarray())



data = np.array(data)
data.shape


result = np.array(result)
result = result.reshape(139,2)



x_train,x_test,y_train,y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)



model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding = 'Same'))
model.add(Conv2D(32, kernel_size=(2, 2),  activation ='relu', padding = 'Same'))


model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))
model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss = "categorical_crossentropy", optimizer='Adamax')
print(model.summary())


y_train.shape


history = model.fit(x_train, y_train, epochs = 30, batch_size = 40, verbose = 1,validation_data = (x_test, y_test))



plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()


def names(number):
    if number==0:
        return 'Its a Tumor'
    else:
        return 'No, Its not a tumor'


from matplotlib.pyplot import imshow
img = Image.open(r"/content/drive/MyDrive/brain_tumor_dataset/no/43 no.jpg")
x = np.array(img.resize((128,128)))
x = x.reshape(1,128,128,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print(str(res[0][classification]*100) + '% Confidence This Is ' + names(classification))


#cat dog classification

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,BatchNormalization,Dropout


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!kaggle datasets download -d salader/dogs-vs-cats


import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()


#generators are very useful to process larger amount of data it divide data into serveral batches

train_ds=tf.keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

validation_ds=tf.keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)



#Normalize the data
def process(image,label):
  image = tf.cast(image/255. ,tf.float32)
  return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers

model_1 = Sequential()

# Convolutional Layer 1
model_1.add(Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu',
                   input_shape=(256,256,3),
                   kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

# Convolutional Layer 2
model_1.add(Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu',
                   kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

# Convolutional Layer 3
model_1.add(Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu',
                    kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

# Flatten Layer
model_1.add(Flatten())

# Dense Layer 1
model_1.add(Dense(128, activation='relu',
                   kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model_1.add(Dropout(0.1))

# Dense Layer 2
model_1.add(Dense(64, activation='relu',
                   kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model_1.add(Dropout(0.1))

# Output Layer
model_1.add(Dense(1, activation='sigmoid'))



model_1.summary()

model_1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


history=model_1.fit(train_ds,epochs=30,validation_data=validation_ds)

import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()


plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

import cv2
test_dog=cv2.imread('/content/sample_data/pexels-svetozar-milashevich-99573-1490908.jpg')
plt.imshow(test_dog)

test_dog.shape
test_dog=cv2.resize(test_dog,(256,256))

test_input=test_dog.reshape((1,256,256,3))
model_1.predict(test_input)

test_cat=cv2.imread('/content/sample_data/cat-1192026_640.jpg')

plt.imshow(test_cat)
test_cat.shape

test_cat=cv2.resize(test_cat,(256,256))
test_inputt=test_cat.reshape((1,256,256,3))
model_1.predict(test_inputt)


# Load and preprocess the image
import tensorflow as tf

def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.cast(img_array / 255.0, tf.float32)  # Normalize to [0, 1]
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
    return img_array
img_array = load_and_preprocess_image('/content/sample_data/cat-1192026_640.jpg')

# Make a prediction
predictions = model_1.predict(img_array)

# Output the prediction
class_prediction = (predictions > 0.5).astype("int32")
print("Predicted class:", "Dog" if class_prediction[0][0] == 1 else "Cat")


# Load and preprocess the image
import tensorflow as tf

def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.cast(img_array / 255.0, tf.float32)  # Normalize to [0, 1]
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
    return img_array
img_array = load_and_preprocess_image('/content/sample_data/pexels-svetozar-milashevich-99573-1490908.jpg')

# Make a prediction
predictions = model_1.predict(img_array)

# Output the prediction
class_prediction = (predictions > 0.5).astype("int32")
print("Predicted class:", "Dog" if class_prediction[0][0] == 1 else "Cat")

import tensorflow as tf
import matplotlib.pyplot as plt

# Load and preprocess the image
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.cast(img_array / 255.0, tf.float32)  # Normalize to [0, 1]
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
    return img, img_array

# Path to the image
image_path = '/content/sample_data/cat-1192026_640.jpg'
img, img_array = load_and_preprocess_image(image_path)

# Make a prediction
predictions = model_1.predict(img_array)

# Output the prediction
class_prediction = (predictions > 0.5).astype("int32")
predicted_class = "Dog" if class_prediction[0][0] == 1 else "Cat"

# Display the image and prediction
plt.imshow(img)
plt.title(f"Prediction: {predicted_class}")
plt.axis('off')  # Hide axes
plt.show()



import tensorflow as tf
import matplotlib.pyplot as plt

# Load and preprocess the image
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.cast(img_array / 255.0, tf.float32)  # Normalize to [0, 1]
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
    return img, img_array

# Path to the image
image_path = '/content/sample_data/pexels-svetozar-milashevich-99573-1490908.jpg'
img, img_array = load_and_preprocess_image(image_path)

# Make a prediction
predictions = model_1.predict(img_array)

# Output the prediction
class_prediction = (predictions > 0.5).astype("int32")
predicted_class = "Dog" if class_prediction[0][0] == 1 else "Cat"

# Display the image and prediction
plt.imshow(img)
plt.title(f"Prediction: {predicted_class}")
plt.axis('off')  # Hide axes
plt.show()

